In [ ]:
# those variables will be updated by Mercury framework
limit = 100
cut_all_after = ['HEVC','HDTV']

In [ ]:
from bs4 import BeautifulSoup  # Getting list of [name, magnectic_links]
import requests
import pandas as pd     #          <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<   Making a Pandas DF
import os  # Check if have a old one and a new one to compare
from operator import itemgetter
from IPython.display import HTML  # Clean UP torrent strings
import pickle  #  SAVE links_pages_data
from datetime import datetime

# Creating Folder to hold the old results to later comparison
folder_path = "./"
files_on_folder = os.listdir(folder_path)
folder_name = "trending_torrents"
if folder_name in files_on_folder:
    print("Found " + folder_name + " using the existing one")
else:
    print("Could not Found " + folder_name + " creating the directory")
    os.mkdir(folder_path + folder_name)
files_on_folder = os.listdir(folder_path)
files_path = folder_path + folder_name + "/"
print("Will save on the folder: " + files_path)

## Acquiring HTML of the PiratesBay's first 35 pages
* Classified by number of leachers (Since number of seeders there is a lot of spoofed torrents)  

"list_of_name_mlinks" is the results list composed of [[name, mlink],[name, mlink],[name, mlink],[name, mlink]...]

In [ ]:
number_of_pages = 35
url_base = "https://pirateproxy.live/browse/200/{}/9"
list_of_name_mlinks = []
for page_num in range(1, number_of_pages + 1):
    r = requests.get(url_base.format(page_num))
    soup = BeautifulSoup(r.text, 'lxml')
    tst = soup.find_all("tr")
    tst2 = tst[1:-1]
    result_page = []
    for it in range(len(tst2)):
#         print("'" + tst2[it].find_all("div", {"class": "detName"})[0].text[1:-1] + "'")
#         print(tst2[1].find_all("a")[3]['href'])
        name = tst2[it].find_all("div", {"class": "detName"})[0].text[1:-1]
        mlink = tst2[1].find_all("a")[3]['href']
        list_of_name_mlinks.append([name, mlink])
print(len(list_of_name_mlinks))

## Creating Pandas Data Frame and saving it to a picklefile

In [ ]:
torrents = pd.DataFrame(list_of_name_mlinks, columns = ["Name", "Mag Link"])
now = datetime.now()
dt_string = now.strftime("%d-%m-%Y_%H:%M:%S")
with open( files_path + 'ptorrent_' + dt_string + '.pkl', 'wb') as f:  # Python 3: open(..., 'wb')
    pickle.dump(torrents, f)
f.close

## Search on the folder for latest to compare with the actual

In [ ]:
files_on_folder = os.listdir(files_path)
to_scrape_folders = []
for item in files_on_folder:
    if item.find("ptorrent_") == 0:
#         print("'" + item[9:item.rfind(".")] + "'")
        date_time_str = item[9:item.rfind(".")]
        date_time_obj = datetime.strptime(date_time_str, '%d-%m-%Y_%H:%M:%S')
        to_scrape_folders.append([files_path + item , date_time_obj])
for it in to_scrape_folders:
    print(it)
print("-------------------------------------------------")
if len(to_scrape_folders) > 1:
    to_scrape_folders = sorted(to_scrape_folders, key=itemgetter(1))  # Put in DateTime ORDER !!!!!!
    for n in range(len(to_scrape_folders)):                           # Print Epochs and time difference betwen them
        print("Epoch from date : " + str(to_scrape_folders[n][1]))
        if to_scrape_folders[n] != to_scrape_folders[-1]:
            print("time passed:>    " + str(to_scrape_folders[n+1][1] - to_scrape_folders[n][1]))
    print("Have More than one to compare !!!!!")
else:
    print("DON`T Have More than one to compare !!!!!")
    raise SystemExit("DON`T Have More than one to compare !!!!!")

## Open both to make comparison

In [ ]:
with open(to_scrape_folders[-2][0], 'rb') as f:  # Python 3: open(..., 'rb')
    print(to_scrape_folders[-2][0])
    product_list_old = pickle.load(f)
f.close
with open(to_scrape_folders[-1][0], 'rb') as f:  # Python 3: open(..., 'rb')
    print(to_scrape_folders[-1][0])
    product_list_new = pickle.load(f)
f.close
to_compare = product_list_new
# to_scrape_folders[-2][1].strftime("%d-%m-%Y_%H:%M:%S")
to_compare[[to_scrape_folders[-2][1].strftime("%d-%m-%Y_%H:%M:%S")]] = None
# [[to_scrape_folders[n+1][1].strftime("%d-%m-%Y_%H:%M:%S")]] = np.nan  # Add a new Column with the name being the datetime 

In [ ]:
to_compare

## Find the latest position (index) of each torrent and compare with the actual

In [ ]:
for index_products in product_list_old.index:    # Add to the comparation table the index from the old one
    to_compare.loc[((to_compare['Name'] == product_list_old.loc[index_products,'Name'])), \
                        to_scrape_folders[-2][1].strftime("%d-%m-%Y_%H:%M:%S")] = int(index_products)
#     to_compare.loc[((to_compare['Name'] == product_list_old.loc[index_products,'Name']) & \
#                         (to_compare['Mag Link'] == product_list_old.loc[index_products,'Mag Link'])), \
#                         to_scrape_folders[-2][1].strftime("%d-%m-%Y_%H:%M:%S")] = int(index_products)

In [ ]:
to_compare

## Compare Actual results with old one and save difference

In [ ]:
to_compare[["diff"]] = None
for index_products in to_compare.index:  # Calculate the Diff column
    old_datetime = to_scrape_folders[-2][1].strftime("%d-%m-%Y_%H:%M:%S")
    if to_compare.loc[index_products, old_datetime] is None:
        to_compare.loc[index_products, old_datetime] = len(to_compare.index)
    if index_products != to_compare.loc[index_products, old_datetime]:
#         print(to_compare.loc[index_products])
        to_compare.loc[index_products, "diff"] = to_compare.loc[index_products, old_datetime]- index_products
    if index_products == to_compare.loc[index_products, old_datetime]:
        to_compare.loc[index_products, "diff"] = 0

In [ ]:
to_compare

## Sort by torrents that had gone up(Trending) and clean name strings

In [ ]:
  # Clean UP torrent strings
result = to_compare.sort_values(by='diff', ascending=False)[:limit] # <<<<<<<<<<<<<<< limit here result 
for it in result.index:  # Regex to clean Name String
    modified_string = False
    name = result.loc[it]['Name']
    for filt in cut_all_after:
        if name.find(filt) != -1:
            modified_string = True
            name = name[:name.find(filt)]
    if modified_string is False:
        name = name
    result.loc[[it], 'Name'] = '<a href="https://www.google.com/search?q={}">{}</a>'.format(name,name)
    result.loc[[it], 'Mag Link'] = '<a href="{}">"Magnectic Link"</a>'.format(result.loc[it]['Mag Link'])
HTML(result.to_html(escape=False))

# FINISH -------------------------------------------